In [1]:
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pylab as plt
%matplotlib inline
import plotly.express as px

In [22]:
df = pd.read_pickle('BBresults.pkl')
pd.set_option('display.max_columns', None)

#creating distance columns
split_df = pd.DataFrame(df['distance'].tolist(), columns=['dist_meters', 'dist_degrees'])
df = pd.concat([df, split_df], axis=1)
dfBB=df[df['channel'] == 'BHZ'] #sorting results for just broadband responses 
dfR=df[df['channel'] == 'EHZ']
dfRS=dfR[dfR['npts']==36001]

In [81]:
#turning it into a loop...
eventlist=dfRS.drop_duplicates(subset=['eventname'])
eventlist=eventlist['eventname']

df_final=pd.DataFrame()

for event in eventlist:
    dfRSevent=pd.DataFrame
    dfRSevent=dfRS[dfRS['eventname']==event]

    dfBBevent=pd.DataFrame
    dfBBevent=dfBB[dfBB['eventname']==event]

    dfevent=pd.DataFrame()

    dfevent['eventname']=dfRSevent["eventname"]
    dfevent['Median Tacer Energy']=dfRSevent["ebbcorrmedtac"]
    dfevent['Station Name']=dfRSevent["station"]
    dfevent['Station Distance']=dfRSevent["dist_degrees"]

    dfevent1=pd.DataFrame()
    dfevent1['eventname']=dfBBevent["eventname"]
    dfevent1['Broadband Median Tacer Energy']=dfBBevent["ebbcorrmedtac"]
    dfevent1['Average Broadband Median Tacer Energy'] = dfevent1["Broadband Median Tacer Energy"].mean()

    dfevent2=pd.merge(dfevent, dfevent1)
    dffinal=dfevent2.drop_duplicates(subset=['Station Name'])
    dffinal = dffinal.drop('Broadband Median Tacer Energy', axis=1)
    dffinal['Normalization']=np.log(dffinal['Median Tacer Energy']/dffinal['Average Broadband Median Tacer Energy'])

    df_final=pd.concat([df_final,dffinal],ignore_index=True)
    df_final.to_pickle("NormMeanErgs.pkl")


In [82]:
df1 = pd.read_pickle('NormMeanErgs.pkl')
df1

,eventname,Median Tacer Energy,Station Name,Station Distance,Average Broadband Median Tacer Energy,Normalization
0,2022052601,1.503072e+15,R9CDF,27.535287,4.357280e+14,1.238248
1,2022052601,8.911960e+14,RAA90,27.304219,4.357280e+14,0.715546
2,2022052601,5.852942e+15,RB079,26.866319,4.357280e+14,2.597681
3,2022052601,1.881909e+16,RC74C,28.636012,4.357280e+14,3.765609
4,2022052601,5.119612e+14,RC98F,28.273964,4.357280e+14,0.161231
...,...,...,...,...,...,...
13110,2021011100,3.959968e+15,RFF1D,55.886105,2.796301e+18,-6.559817
13111,2021011100,2.791159e+15,S0306,24.280720,2.796301e+18,-6.909596
13112,2021011100,1.068487e+18,S1108,58.186981,2.796301e+18,-0.962054
13113,2021011100,1.987318e+14,S4458,23.856543,2.796301e+18,-9.551852


In [84]:
distance = df1['Station Distance'].tolist()
normalization = df1['Normalization'].tolist()



fig1 = px.scatter(x=distance, y=normalization, title='All Events, RS Stations Normalized by BB Mean', labels={"x":"Distance in Degrees", "y":"Log Normalization of Median Tacer Energy"})
fig1.update_yaxes(range = [-10,10])
fig1